In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('C://work/dev/python/progs/texts/sec_bert/')

# Объявления

In [3]:
import pandas as pd
import numpy as np
import phik
from itertools import chain


from sklearn.metrics import classification_report

SEED = 0
np.random.seed(SEED)
import joblib
from sklearn.metrics import average_precision_score

In [7]:
from ruamel.yaml import YAML
conf = YAML().load(open('params.yaml'))

# Загружаем

In [8]:
import pandas as pd
import numpy as np
import joblib
import json
import click
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import ClassifierChain
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import log_loss, roc_auc_score, average_precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [10]:
feat_data = pd.read_csv(conf['feat_eng']['feat_final_fn'])
mlb = joblib.load(conf['prep_text']['mlb_fn'])
data = pd.read_csv(conf['feat_gen']['data_fn'])

data['target'] = data['target'].map(lambda x: eval(x))

data['labels'] = data['labels'].map(lambda x: eval(x))

with open(conf['get_data']['label2tactic_fn'], 'rt') as f_r:
        label2tactic = json.load(f_r)

wrap_class = ClassifierChain

tr_idx = data.query('split=="tr"').index
val_idx = data.query('split=="val"').index
ts_idx = data.query('split=="ts"').index



# Генерим пары

In [11]:
col_l = [col for col in feat_data.columns]

feat_data = feat_data[col_l]

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize

def custom_tok(x):
    return word_tokenize(x)
    
vec = TfidfVectorizer(tokenizer=custom_tok, binary=conf['feat_gen']['binary'], 
                      max_features=conf['feat_gen']['max_features'], min_df=conf['feat_gen']['min_df'], 
                      max_df=conf['feat_gen']['max_df'], ngram_range=(1,3))
vec.fit(data.query('split=="tr"')['prep_text'])

pairs_data = pd.DataFrame(vec.transform(data['prep_text']).toarray(), columns=vec.get_feature_names_out(), index=data.index)


C:\tools\Anaconda3\envs\pytorch_env\lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Атаки

как передать папку в кластер spark c загружаемыми модулями и другими файлами. Например, хочу передать папку nltk_data во все ноды, для работы в уоркерах

## defence_evasion

In [16]:
cls = 'defense-evasion'

cls_filter = (data['labels'].map(lambda x: cls in x))

### разность слов

In [18]:
cls_words = set(chain(*[it.split() for it in data.loc[(data.index.isin(tr_idx))&cls_filter, 'prep_text'].tolist()]))
all_words = set(chain(*[it.split() for it in data.loc[(data.index.isin(tr_idx))&(~cls_filter), 'prep_text'].tolist()]))

In [20]:
feat_data[ 'y'] = 0
feat_data.loc[cls_filter, 'y'] = 1

feat_data['y'] = feat_data['y'].astype(int)

In [21]:
from sklearn.tree import DecisionTreeClassifier

cls = DecisionTreeClassifier(max_depth=10, random_state=conf['seed'])

cls.fit(feat_data.drop(columns='y').loc[tr_idx], feat_data['y'].loc[tr_idx])

DecisionTreeClassifier(max_depth=10, random_state=0)

In [22]:
y_p = cls.predict(feat_data.drop(columns='y').loc[val_idx])


print(classification_report(feat_data['y'].loc[val_idx], y_p))

              precision    recall  f1-score   support

           0       0.82      0.96      0.89      1810
           1       0.77      0.36      0.49       590

    accuracy                           0.82      2400
   macro avg       0.80      0.66      0.69      2400
weighted avg       0.81      0.82      0.79      2400



In [26]:
imps_df = pd.DataFrame({'feat':feat_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
    .sort_values(by='imp', ascending=False)
imps_df.head(10)

,feat,imp
11,delet,0.143068
4,dll,0.101044
27,decrypt,0.092897
30,bypass,0.078620
33,obfusc,0.075861
41,legitim,0.067337
73,modifi,0.065279
10,inject,0.061541
79,payload,0.039942
86,sign,0.036097


In [36]:
idx = np.where(imps_df['imp'].cumsum()/imps_df['imp'].sum()>thresh)[0].min()

In [43]:
[it for it in imps_df.loc[imps_df.index[:idx], 'feat'].tolist() if not it in col_l]

[]

## функция

In [45]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize

def custom_tok(x):
    return word_tokenize(x)
    
vec = TfidfVectorizer(tokenizer=custom_tok, binary=conf['feat_gen']['binary'], 
                      max_features=conf['feat_gen']['max_features'], min_df=conf['feat_gen']['min_df'], 
                      max_df=conf['feat_gen']['max_df'], ngram_range=(1,3))
vec.fit(data.query('split=="tr"')['prep_text'])

feat_full_data = pd.DataFrame(vec.transform(data['prep_text']).toarray(), columns=vec.get_feature_names_out(), index=data.index)

C:\tools\Anaconda3\envs\pytorch_env\lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [104]:
%%time
def add_new_feat(data, feat_full_data, cls, feat_thresh = 0.8):    
    
    cls_filter = (data['labels'].map(lambda x: cls in x))
    
    feat_full_data[ 'y'] = 0
    feat_full_data.loc[cls_filter, 'y'] = 1
    
    feat_full_data['y'] = feat_full_data['y'].astype(int)
    
    cls = DecisionTreeClassifier(max_depth=10, random_state=conf['seed'])
    
    cls.fit(feat_full_data.drop(columns='y').loc[tr_idx], feat_full_data['y'].loc[tr_idx])
    
    imps_df = pd.DataFrame({'feat':feat_full_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
        .sort_values(by='imp', ascending=False)
    
    idx = np.where(imps_df['imp'].cumsum()/imps_df['imp'].sum()>thresh)[0].min()
    
    return [it for it in imps_df.loc[imps_df.index[:idx], 'feat'].tolist() if not it in col_l]

feat_d = {}
for cls in mlb.classes_:
    feat_d[cls] = add_new_feat(data, feat_full_data, cls)

CPU times: total: 4min 48s
Wall time: 4min 52s


In [106]:
from itertools import chain

new_col_l = list(chain(*feat_d.values()))

['hidden',
 'disabl',
 'run key',
 'token',
 'ad',
 'connect',
 'use powershel',
 'attempt get user',
 'vbscript',
 'run key',
 'web shell',
 'ad',
 'connect',
 'encod',
 'https',
 'archiv',
 'upload file from',
 'document',
 'pass the',
 'remot desktop',
 'use rdp',
 'for later movement',
 'move later',
 'vnc',
 'network share',
 'use ssh',
 'share citat',
 'connect',
 '2021',
 '2011',
 'from',
 'mimikatz',
 'determin',
 'their oper',
 'regist domain',
 'has acquir',
 'for oper',
 'use dure target',
 'develop',
 'host malici',
 'media account',
 'and/or initi',
 'employe',
 'iran',
 'email address',
 'against',
 'citat googl',
 'for access',
 'septemb 2020',
 'citat clearski lazarus',
 'mbr',
 'ransom',
 'denial servic',
 'overwrit',
 'mine',
 'stop',
 'encrypt file',
 'destroy',
 'goal',
 'destroy 2018',
 'gain initi',
 'contain',
 'e-mail',
 'hole',
 'use domain',
 'deliv via',
 'gain access',
 'for initi',
 'network citat',
 'user',
 '2021',
 'has target',
 'vpns',
 'gru',
 'back',

In [107]:
feat_d

{'defense-evasion': ['hidden', 'disabl'],
 'privilege-escalation': ['run key', 'token', 'ad', 'connect'],
 'execution': ['use powershel', 'attempt get user', 'vbscript'],
 'persistence': ['run key', 'web shell', 'ad', 'connect'],
 'command-and-control': ['encod', 'https'],
 'collection': ['archiv', 'upload file from', 'document'],
 'lateral-movement': ['pass the',
  'remot desktop',
  'use rdp',
  'for later movement',
  'move later',
  'vnc',
  'network share',
  'use ssh',
  'share citat',
  'connect',
  '2021',
  '2011'],
 'credential-access': ['from', 'mimikatz'],
 'discovery': ['determin'],
 'resource-development': ['their oper',
  'regist domain',
  'has acquir',
  'for oper',
  'use dure target',
  'develop',
  'host malici',
  'media account'],
 'reconnaissance': ['and/or initi',
  'employe',
  'iran',
  'email address',
  'against',
  'citat googl',
  'for access',
  'septemb 2020',
  'citat clearski lazarus'],
 'impact': ['mbr',
  'ransom',
  'denial servic',
  'overwrit',
  

In [98]:
cls = 'impact'
cls_filter = (data['labels'].map(lambda x: cls in x))

feat_full_data[ 'y'] = 0
feat_full_data.loc[cls_filter, 'y'] = 1

feat_full_data['y'] = feat_full_data['y'].astype(int)

In [99]:
cls = DecisionTreeClassifier(max_depth=1, random_state=conf['seed'])

cls.fit(feat_full_data.drop(columns='y').loc[tr_idx], feat_full_data['y'].loc[tr_idx])

imps_df = pd.DataFrame({'feat':feat_full_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
    .sort_values(by='imp', ascending=False)

idx = np.where(imps_df['imp'].cumsum()/imps_df['imp'].sum()>thresh)[0].min()

[it for it in imps_df.loc[imps_df.index[:idx], 'feat'].tolist() if not it in col_l]

[]

In [100]:
idx = np.where(imps_df['imp'].cumsum()/imps_df['imp'].sum()>0.99)[0].min()
idx

0

In [101]:
imps_df

,feat,imp
2430,delet,1.0
0,.,0.0
5142,octob,0.0
5140,octo tempest,0.0
5139,octo,0.0
...,...,...
2562,docker,0.0
2561,dns tunnel,0.0
2560,dns request,0.0
2559,dns for,0.0


In [93]:
[it for it in imps_df.loc[imps_df.index[:idx], 'feat'].tolist() if not it in col_l]

[]

In [86]:
[it for it in imps_df.loc[imps_df.index[:idx], 'feat'].tolist() if not it in col_l]

['hidden',
 'disabl',
 'modifi the',
 'load and',
 '2022',
 'victim system',
 'web',
 'novemb',
 'warzonerat',
 'dure oper',
 'also',
 'code',
 '2019 citat',
 'them',
 'vssadmin',
 'browser',
 'function',
 'microsoft',
 'user',
 'mfa',
 'host citat',
 'implant',
 'and then',
 'store',
 'ssl',
 'lazarus group malwar',
 'process inject',
 'powerduk novemb 2016',
 'fin13',
 'and load',
 'citat mandiant fin12']

## reconnaissance

Найдем, есть ли слова, которые хорошо определяют данный тип атаки, для этого:

- построим облако разницы

In [14]:
cls = 'reconnaissance'

In [15]:
# cls_filter = (data.index.isin(tr_idx))&(data['labels'].map(lambda x: 'reconnaissance' in x))
cls_filter = (data['labels'].map(lambda x: cls in x))

### разность слов

In [16]:
cls_words = set(chain(*[it.split() for it in data.loc[(data.index.isin(tr_idx))&cls_filter, 'prep_text'].tolist()]))
all_words = set(chain(*[it.split() for it in data.loc[(data.index.isin(tr_idx))&(~cls_filter), 'prep_text'].tolist()]))

Нет явных слов, которые бы отсутствовали в общем сете

### классификатор

#### по словам

In [18]:
feat_data[ 'y'] = 0
feat_data.loc[cls_filter, 'y'] = 1

feat_data['y'] = feat_data['y'].astype(int)

In [19]:
from sklearn.tree import DecisionTreeClassifier

cls = DecisionTreeClassifier(max_depth=10, random_state=conf['seed'])

cls.fit(feat_data.drop(columns='y').loc[tr_idx], feat_data['y'].loc[tr_idx])

DecisionTreeClassifier(max_depth=10, random_state=0)

In [20]:
y_p = cls.predict(feat_data.drop(columns='y').loc[val_idx])


print(classification_report(feat_data['y'].loc[val_idx], y_p))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2378
           1       0.62      0.32      0.42        25

    accuracy                           0.99      2403
   macro avg       0.80      0.66      0.71      2403
weighted avg       0.99      0.99      0.99      2403



In [21]:
pd.DataFrame({'feat':feat_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
    .sort_values(by='imp', ascending=False)\
    .head(10)

,feat,imp
81,search,0.185060
53,phish,0.162019
19,target,0.108920
97,for,0.107981
105,lazarus,0.045361
100,collect,0.039779
64,gather,0.037616
73,about,0.035988
98,server,0.034175
56,credenti,0.032908


#### по парам слов

In [22]:
pairs_data[ 'y'] = 0
pairs_data.loc[cls_filter, 'y'] = 1

pairs_data['y'] = pairs_data['y'].astype(int)

NameError: name 'pairs_data' is not defined

In [16]:
from sklearn.tree import DecisionTreeClassifier

cls = DecisionTreeClassifier(max_depth=10, random_state=conf['seed'])

cls.fit(pairs_data.drop(columns='y').loc[tr_idx], pairs_data['y'].loc[tr_idx])

DecisionTreeClassifier(max_depth=10, random_state=0)

In [17]:
y_p = cls.predict(pairs_data.drop(columns='y').loc[val_idx])
print(classification_report(pairs_data['y'].loc[val_idx], y_p))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1284
           1       0.50      0.33      0.40         6

    accuracy                           1.00      1290
   macro avg       0.75      0.67      0.70      1290
weighted avg       0.99      1.00      0.99      1290



In [18]:
pd.DataFrame({'feat':pairs_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
    .sort_values(by='imp', ascending=False)\
    .head(10)

,feat,imp
5645,reveal for,0.485407
4018,lapsus has,0.075006
3423,gru unit,0.062087
7257,threat octob,0.059481
5788,search websit,0.045727
2761,exchang maraud,0.025671
4096,lili has,0.022743
1697,citat rostovcev,0.022728
2550,effort citat,0.022154
4205,lotus februari,0.019061


In [20]:
word = 'reveal for'

#### слово для тактики


In [21]:
with pd.option_context('display.max_colwidth', 500):
    display(data.explode('labels').query('labels=="reconnaissance" and prep_text.str.contains(@word)')['prep_text'].iloc[0])

' may gather about the victim host that can use dure target about host may includ varieti detail includ data name assign etc well specif regard it oper system languag etc. may gather this various way such direct collect action via activ scan phish for may also compromis site then includ malici content design collect host from visitor citat att scanbox about host may also expos via onlin other access data set social media search websit gather this may reveal for other form search open search open technic databas resourc develop obtain and/or initi access suppli chain compromis extern remot servic'

#### распределение тактик с конкретным словом


In [22]:
data.explode('labels').assign(has_sent = lambda x: x['prep_text'].str.contains(word)).pivot_table(index='labels', values='has_sent', aggfunc=['count', 'sum', 'mean'
]).sort_values(by=('mean', 'has_sent'), ascending=False)

,count,sum,mean
,has_sent,has_sent,has_sent
labels,,,
reconnaissance,152,37,0.243421
defense-evasion,5688,1,0.000176
collection,1357,0,0.000000
command-and-control,2773,0,0.000000
credential-access,1050,0,0.000000
discovery,3211,0,0.000000
execution,2731,0,0.000000
exfiltration,401,0,0.000000


<div class='alert alert-info'>
    - сочетание reveal for имеет хорошую распознавательную способность для тактики reconnaissance
</div>

## impact

Найдем, есть ли слова, которые хорошо определяют данный тип атаки, для этого:

- построим облако разницы

In [43]:
cls = 'impact'

In [24]:
# cls_filter = (data.index.isin(tr_idx))&(data['labels'].map(lambda x: 'reconnaissance' in x))
cls_filter = (data['labels'].map(lambda x: cls in x))

### разность слов

In [25]:
cls_words = set(chain(*[it.split() for it in data.loc[(data.index.isin(tr_idx))&cls_filter, 'prep_text'].tolist()]))
all_words = set(chain(*[it.split() for it in data.loc[(data.index.isin(tr_idx))&(~cls_filter), 'prep_text'].tolist()]))

In [26]:
cls_words.difference(all_words)

{'-class',
 '.avo',
 '.avos2',
 '.avoslinux',
 '.bk',
 '.clop',
 '.cuba',
 '.disabl',
 '.enc',
 '.lock64',
 '.ryk',
 '.seth',
 '/for=c',
 '1.5mb',
 '10gbps',
 '132mb',
 '15,000',
 '1gbps',
 '2048-bit',
 '4.9mb',
 '500mb',
 '51,200',
 'aes-rsa',
 'bart',
 'bbc-ronin',
 'bind9',
 'bitlock',
 'boot-up',
 'bootwreck',
 'butcher',
 'catalyst',
 'chernobyl',
 'cipher.ex',
 'cryptopp',
 'crytox',
 'cybleinc',
 'cynet',
 'degrad',
 'disast',
 'discomfort',
 'diskshadow',
 'distrust',
 'disturb',
 'doj-dprk',
 'dyepack',
 'fbi-bec',
 'fsisac',
 'gobotkr',
 'govt',
 'gpt',
 'hacktiv',
 'heap',
 'httpflood',
 'immens',
 'inoper',
 'intimid',
 'ioctl',
 'koto',
 'lolmin',
 'magnitud',
 'mbed',
 'memcach',
 'memcrash',
 'mgf1',
 'noberus',
 'non-system',
 'ntp',
 'ntru',
 'passwd',
 'pig',
 'ras-4096',
 'reflector',
 'reformat',
 'rmshutdown',
 'rsa-1028',
 'rsa-oaep',
 'satur',
 'scm',
 'senat',
 'siemen',
 'siprotec',
 'sosemanuk',
 'ssldos',
 'swift',
 'taunt',
 'usnyag',
 'w95.cih',
 'winr',
 '

будто есть характерные слова для этой тактики

### классификатор

#### по словам

In [27]:
feat_data[ 'y'] = 0
feat_data.loc[cls_filter, 'y'] = 1

feat_data['y'] = feat_data['y'].astype(int)

In [28]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=10, random_state=conf['seed'])

clf.fit(feat_data.drop(columns='y').loc[tr_idx], feat_data['y'].loc[tr_idx])

DecisionTreeClassifier(max_depth=10, random_state=0)

In [29]:
y_p = clf.predict(feat_data.drop(columns='y').loc[val_idx])


print(classification_report(feat_data['y'].loc[val_idx], y_p))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1280
           1       0.60      0.30      0.40        10

    accuracy                           0.99      1290
   macro avg       0.80      0.65      0.70      1290
weighted avg       0.99      0.99      0.99      1290



In [31]:
pd.DataFrame({'feat':feat_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
    .sort_values(by='imp', ascending=False)\
    .head(10)


,feat,imp
4857,citat,0.324068
12805,shadow,0.098715
10822,overwrit,0.081288
5794,denial,0.052746
5833,destroy,0.039232
15407,wallpap,0.030064
13657,swift,0.027821
11479,prolong,0.027816
9719,mine,0.024473
13440,stop,0.023186


#### по парам слов

In [32]:
pairs_data[ 'y'] = 0
pairs_data.loc[cls_filter, 'y'] = 1

pairs_data['y'] = pairs_data['y'].astype(int)

In [33]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=10, random_state=conf['seed'])

clf.fit(pairs_data.drop(columns='y').loc[tr_idx], pairs_data['y'].loc[tr_idx])

DecisionTreeClassifier(max_depth=10, random_state=0)

In [34]:
y_p = clf.predict(pairs_data.drop(columns='y').loc[val_idx])
print(classification_report(pairs_data['y'].loc[val_idx], y_p))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1280
           1       0.75      0.30      0.43        10

    accuracy                           0.99      1290
   macro avg       0.87      0.65      0.71      1290
weighted avg       0.99      0.99      0.99      1290



In [35]:
pd.DataFrame({'feat':pairs_data.drop(columns='y').columns, 'imp':cls.feature_importances_})\
    .sort_values(by='imp', ascending=False)\
    .head(10)

,feat,imp
10,/all /quiet,0.113068
2249,denial servic,0.109864
2374,disk structur,0.094003
6208,stop servic,0.065820
5972,shadow copi,0.058831
2631,encrypt file,0.056200
5515,reboot the,0.050349
1592,citat fireey,0.043120
3777,integr the,0.035288
1333,can encrypt,0.032478


In [76]:
word = 'overwrit'

#### слово для тактики


In [73]:
with pd.option_context('display.max_colwidth', 500):
    display(data.explode('labels').query('labels==@cls and prep_text.str.contains(@word)')['prep_text'].iloc[0])

' may delet remov built-in data and turn off servic design aid the recoveri corrupt system prevent recoveri citat talo olymp destroy 2018 citat fireey wannacri 2017 this may deni access avail backup and recoveri option oper system may contain featur that can help fix corrupt system such backup catalog volum shadow copi and automat repair featur may disabl delet system recoveri featur augment the effect data and data encrypt for impact citat talo olymp destroy 2018 citat fireey wannacri 2017 may disabl recoveri then corrupt backup citat number nativ window util have been use disabl delet system recoveri featur code /code can use delet all volum shadow copi system code delet shadow /all /quiet /code window manag can use delet volum shadow copi code wmic shadowcopi delet /code code /code can use delet the window backup catalog code delet catalog -quiet /code code /code can use disabl automat window recoveri featur modifi boot data code /set default bcdedit /set default /code code /code ca

In [74]:
data.explode('labels').query('labels==@cls and prep_text.str.contains(@word)')

,sentence,labels,prep_text,target,split,threat_words
625,Adversaries may delete or remove built-in data...,impact,may delet remov built-in data and turn off se...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
2878,[HermeticWiper] has the ability to stop the Vo...,impact,has the abil stop the volum shadow copi servi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14058,[Black Basta] can delete shadow copies using v...,impact,black basta can delet shadow copi use citat m...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14059,[Ragnar Locker] can delete volume shadow copie...,impact,ragnar locker can delet volum shadow copi use...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14061,[WastedLocker] can delete shadow volumes.(Cita...,impact,can delet shadow volum citat symantec june 20...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14062,[H1N1] disable recovery options and deletes sh...,impact,h1n1 disabl recoveri option and delet shadow ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14063,[RobbinHood] deletes shadow copies to ensure t...,impact,robbinhood delet shadow copi ensur that all t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14064,[Ryuk] has used <code>vssadmin Delete Shadows ...,impact,ryuk has use code vssadmin delet shadow /all ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14065,[DarkWatchman] can delete shadow volumes using...,impact,can delet shadow volum use code /code citat p...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]
14066,[EKANS] removes backups of Volume Shadow Copie...,impact,ekan remov backup volum shadow copi disabl an...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",tr,[]


#### распределение тактик с конкретным словом


In [77]:
data.explode('labels').assign(has_sent = lambda x: x['prep_text'].str.contains(word)).pivot_table(index='labels', values='has_sent', aggfunc=['count', 'sum', 'mean'
]).sort_values(by=('mean', 'has_sent'), ascending=False)

,count,sum,mean
,has_sent,has_sent,has_sent
labels,,,
impact,264,24,0.090909
persistence,1863,7,0.003757
defense-evasion,5688,19,0.003340
privilege-escalation,2314,5,0.002161
execution,2731,4,0.001465
credential-access,1050,1,0.000952
collection,1357,1,0.000737
discovery,3211,2,0.000623


<div class='alert alert-info'>
    - для тактики impact есть характерные сочетания:
        - /all /quiet	0.113068
        - denial servic	0.109864
        - disk structur	0.094003
        - stop servic	0.065820
        - shadow copi	0.058831

        - shadow
        - overwrit

</div>